In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

import scipy.special as ss

In [2]:
def beta(x):
    return np.sqrt(2/np.pi)*np.exp(-x**2)/ss.erfc(x)

def covariance(X,A):
    'input: X shape (days*years,lat*lon*field), A shape (days*year,)'
    XAs = np.concatenate([X,A.reshape(-1,1)], axis=-1)
    XAs_cov = np.cov(XAs.T)
    sigma_XA = XAs_cov[-1,:-1]
    sigma_AA = XAs_cov[-1,-1]
    sigma_XX = XAs_cov[:-1,:-1]
    return sigma_XX,sigma_XA,sigma_AA

def composite_GA(sigma_AA,sigma_XA,a):
    return beta(a/np.sqrt(2*sigma_AA))*sigma_XA/np.sqrt(sigma_AA)

def composite_data(X,A,a):
    return np.mean(X[A>=a,:], axis=0)

## Test on synthetic data

In [24]:
N = 10000
X = np.random.normal(0,1, (N,2))
X[:,1] += 10*X[:,0]
A = 2*X[:,0] + np.random.normal(0,1,N)

# A = np.random.normal(0,2,N)
# X = np.ones((N,4))
# X[:,0] = 0.2*A + 0.1*np.random.normal(0,1,N)
# X[:,1] = 0.5*A**2 + A + np.random.normal(0,1,N)
# X[:,2] = np.random.normal(0,1,N)
# X[:,3] = 0.2*A + np.sin(0.3*np.random.normal(0,1,N))

In [25]:
# normalize data
X = (X - np.mean(X, axis=0))/np.std(X, axis=0)
# A = (A - np.mean(A))/np.std(A)

sigma_XX,sigma_XA,sigma_AA = covariance(X,A)

In [5]:
i_XX = np.linalg.inv(sigma_XX)

In [26]:
C = sigma_XA
C /= np.sqrt(np.sum(C**2))
C

array([0.70917866, 0.70502882])

In [30]:
epsilon = 0.001
M = np.linalg.inv(sigma_XX + epsilon*np.identity(sigma_XX.shape[0])) @ sigma_XA
M /= np.sqrt(np.sum(M**2))
M

array([0.99996409, 0.00847465])

In [ ]:
sigma_AA - sigma_XA @ i_XX @ sigma_XA

In [ ]:
F = X @ (i_XX @ sigma_XA)
F.shape

In [ ]:
plt.figure()
plt.scatter(F, A)

In [ ]:
XAs = np.concatenate([X,A.reshape(-1,1)], axis=-1)
Sigma = np.cov(XAs.T)
Lambda = np.linalg.inv(Sigma)

In [ ]:
Lambda[-1,-1], 1/(sigma_AA - sigma_XA @ i_XX @ sigma_XA)

In [ ]:
-Lambda[:-1,-1]/Lambda[-1,-1], i_XX @ sigma_XA

In [ ]:
plt.close(1)
fig, ax = plt.subplots(num=1, figsize=(9,6))

plt.hist2d(X[:,0],A,bins=100)

fig.tight_layout()

In [ ]:
ap = np.linspace(np.min(A), np.max(A), 1001) # array of thresholds
apd = np.stack([ap]*X.shape[1]).T # broadcasted ap
cgs = composite_GA(sigma_AA,sigma_XA,apd) # gaussian composites
cds = np.stack([composite_data(X,A,a) for a in ap]) # data composites
cgs.shape, cds.shape

In [ ]:
a = 1

comp_g = composite_GA(sigma_AA,sigma_XA,a)
comp_d = composite_data(X,A,a)

comp_d, comp_g

In [ ]:
idx = 1

plt.close(2)
fig, ax = plt.subplots(num=2, figsize=(9,6))

# density of the data
plt.hist2d(X[:,idx],A,bins=100)

# axes
plt.axvline(0, color='gray')
plt.axhline(0, color='gray')

# linear fits
xis = np.linspace(np.min(X[:,idx]), np.max(X[:,idx]), 3)
plt.plot(xis, xis*sigma_XA[idx]/sigma_XX[idx,idx], color='lime', label=r'$A = \alpha X$') # A vs X
plt.plot(ap*sigma_XA[idx]/sigma_AA, ap, color='yellow', label=r'$X = \xi A$') # X vs A

# threshold
plt.axhline(a, color='red', label=r'$a$')

# data composites
plt.plot(cds[:,idx], ap, color='orange', label=r'$C_D(a)$')
plt.axvline(comp_d[idx], color='orange', linestyle='dashed')

# gaussian composites
plt.plot(cgs[:,idx], ap, color='white', label=r'$C_G(a)$')
plt.axvline(comp_g[idx], color='white', linestyle='dashed')

plt.legend()

plt.xlabel('X')
plt.ylabel('A')

fig.tight_layout()